In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob
from sklearn.decomposition import PCA
#import umap

from tqdm import tqdm
import sleap

import sklearn.experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 
from scipy.io import loadmat
import scipy
import scipy.ndimage
# 

import numba
from numba import jit
import parmap

# 
from track import track
from convert import convert
from ethogram import ethogram

Autosaving every 180 seconds


In [23]:
############################################
########### LOAD AND SPLIT DATA ############
############################################

# Load sleap pkg file containing labeled data and images
labels = sleap.load_file('/media/cat/1TB/dan/cohort1/sleap/day001.pkg.slp')

print ("labels.count: ", labels.count)

for k in range(len(labels.videos)):
    #print (k, labels.labels[k])
    print (k, labels.videos[k])

labels.count:  <bound method Sequence.count of Labels(labeled_frames=4434, videos=23, skeletons=1, tracks=4)>
0 Video(filename=/media/cat/1TB/dan/cohort1/sleap/day001.pkg.slp, shape=(146, 1024, 1280, 3), backend=HDF5Video)
1 Video(filename=/media/cat/1TB/dan/cohort1/sleap/day001.pkg.slp, shape=(181, 1024, 1280, 3), backend=HDF5Video)
2 Video(filename=/media/cat/1TB/dan/cohort1/sleap/day001.pkg.slp, shape=(200, 1024, 1280, 3), backend=HDF5Video)
3 Video(filename=/media/cat/1TB/dan/cohort1/sleap/day001.pkg.slp, shape=(200, 1024, 1280, 3), backend=HDF5Video)
4 Video(filename=/media/cat/1TB/dan/cohort1/sleap/day001.pkg.slp, shape=(200, 1024, 1280, 3), backend=HDF5Video)
5 Video(filename=/media/cat/1TB/dan/cohort1/sleap/day001.pkg.slp, shape=(200, 1024, 1280, 3), backend=HDF5Video)
6 Video(filename=/media/cat/1TB/dan/cohort1/sleap/day001.pkg.slp, shape=(200, 1024, 1280, 3), backend=HDF5Video)
7 Video(filename=/media/cat/1TB/dan/cohort1/sleap/day001.pkg.slp, shape=(200, 1024, 1280, 3), backe

In [ ]:

       #
labels_train = labels.extract(idx_train, copy=True)
labels_val = labels.extract(idx_val, copy=True)
labels_test = labels.extract(idx_test, copy=True)


# Generate a 0.8/0.1/0.1 split
labels_train, labels_test = labels.split(n=0.9)
# labels_val, labels_test = labels_val_test.split(n=0.5)


# # Save with images
# labels_train.save("train.pkg.slp", with_images=True)
# #labels_val.save("val.pkg.slp", with_images=True)
# labels_test.save("test.pkg.slp", with_images=True)


# # Reload and check the grond truh labels in each of these packages:
# labels = sleap.load_file("train.pkg.slp")
# labels....

In [6]:
import numpy as np
from sklearn.model_selection import KFold
X = np.arange(100)
y = np.zeros(100)
kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(X)

print(kf)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    

KFold(n_splits=10, random_state=None, shuffle=True)
TRAIN: [ 0  2  3  4  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 24 25 26 27
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 44 45 46 47 48 50 51 52 53 54
 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 74 75 76 77 78 80
 81 82 83 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99] TEST: [ 1  5 17 23 28 43 49 73 79 84]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24
 25 27 28 29 30 31 32 33 34 36 37 38 39 40 42 43 44 45 46 47 48 49 50 51
 52 54 55 56 57 58 59 60 61 62 63 64 65 68 69 70 71 72 73 74 75 76 77 78
 79 80 81 82 83 84 85 86 87 88 89 91 92 93 94 95 97 98] TEST: [13 26 35 41 53 66 67 90 96 99]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 16 17 19 20 21 22 23 24 25 26
 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
 51 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 69 70 71 72 73 75 76 77
 78 79 80 81 82 84 85 86 87 88 89 90 91 93 94 96 98 99] TEST: [11 15 18 52 68 74 8

In [11]:
#############################
### CUSTOM SPLIT FUNCTION ###
#############################

# load sleap label file
# labels = sleap.load_file(...)

#
n_tot_labels = 500      # total # of human labels

n_labels = 100          # from 100 to 500 labels 

n_validation = 10

val_ratio = 0.9

n_train = int(n_labels*val_ratio)
n_test = int(n_labels*(1-val_ratio))
    
# shuffled res
idx = np.random.choice(np.arange(n_tot_labels),n_tot_labels, replace=False)    

# 
for k in range(n_validation):
    idx_train = idx[n_test*k:n_test*k+n_train]
    idx_test = idx[n_test*k+n_train:n_test*(k+1)+n_train]
    print ("idx_train:", idx_train)
    print ("idx_test: ", idx_test)
    
#     # 
#     labels_train = labels.extract(idx_train, copy=True)
#     labels_test = labels.extract(idx_test, copy=True)

#     # Save with images
#     labels_train.save("train_"+str(ctr)+".pkg.slp", with_images=True)
#     labels_test.save("test_"+str(ctr)+".pkg.slp", with_images=True)

    
    

idx_train: [411 271 127 200   2 422 360 106 368 392 464 400 341  12 386 166  49 245
  88 218 275 111  78  77  95 205 138 348  74  57 153 165 281 171 371   5
  45  43   0   7  65  92 300 316 402 337 370 135 454 363 235 163  22 263
  64 314 190  70 322 162 172  97  68   3 262 351 320 404 486 283 497 425
 108 330 434 141 164 357 498 143 145 173 239 419   1 352 225  20 113 313]
idx_test:  [ 25 397 250 191 215  83  18 212 276]
idx_train: [392 464 400 341  12 386 166  49 245  88 218 275 111  78  77  95 205 138
 348  74  57 153 165 281 171 371   5  45  43   0   7  65  92 300 316 402
 337 370 135 454 363 235 163  22 263  64 314 190  70 322 162 172  97  68
   3 262 351 320 404 486 283 497 425 108 330 434 141 164 357 498 143 145
 173 239 419   1 352 225  20 113 313  25 397 250 191 215  83  18 212 276]
idx_test:  [267 181  90  37 347   4 439 323 130]
idx_train: [ 88 218 275 111  78  77  95 205 138 348  74  57 153 165 281 171 371   5
  45  43   0   7  65  92 300 316 402 337 370 135 454 363 235 163

In [ ]:
#########################################################
######### TRAIN SLEAP ON A TRAINING PKG DATASET #########
#########################################################

# comamnd line run:
command = 'sleap-train '+'train.pkg.slp'

os.command(command)

In [ ]:
###########################################
######### PREDICT ON THE TEST SET #########
###########################################

# comamnd line run:
command = 'sleap-train '+'test.pkg.slp'

os.command(command)



In [ ]:
###########################################
######### PREDICT ON THE TEST SET #########
###########################################

# Reload and check the grond truh labels in each of these packages:
labels = sleap.load_file("train.pkg.slp")
labels....




In [ ]:
###############################################################
### REDEFINE SPLIT FUNCTION TO ALLOW FOR PROPER XVALIDATION ###
###############################################################

def split( 
         self, n: Union[float, int], copy: bool = True 
     ) -> Tuple["Labels", "Labels"]: 
         """Split labels randomly. 
  
         Args: 
             n: Number or fraction of elements in the first split. 
             copy: If `True` (the default), return copies of the labels. 
  
         Returns: 
             A tuple of `(labels_a, labels_b)` where both are `sleap.Labels` instances 
             subsampled from these labels. 
  
         Notes: 
             If there is only 1 labeled frame, this will return two copies of the same 
             labels. For `len(labels) > 1`, splits are guaranteed to be mutually 
             exclusive. 
  
         Example: 
             You can generate multiple splits by calling this repeatedly: 
  
             ```py 
             # Generate a 0.8/0.1/0.1 train/val/test split. 
             labels_train, labels_val_test = labels.split(n=0.8) 
             labels_val, labels_test = labels_val_test.split(n=0.5) 
             ``` 
         """ 
         if len(self) == 1: 
             if copy: 
                 return self.copy(), self.copy() 
             else: 
                 return self, self 
  
         # Split indices. 
         if type(n) != int: 
             n = round(len(self) * n) 
         n = max(min(n, len(self) - 1), 1) 
         idx_a, idx_b = train_test_split(list(range(len(self))), train_size=n) 
  
         return self.extract(idx_a, copy=copy), self.extract(idx_b, copy=copy) 